### <font color = 'purple'>"Repeat is the best medicine for"을 입력받으면 "is the best medicine for memory"를 출력하는 RNN</font>

# 1. 훈련 데이터 전처리

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()
print(sentence)

['Repeat', 'is', 'the', 'best', 'medicine', 'for', 'memory']


In [3]:
vocab = list(set(sentence))
print(vocab)

['Repeat', 'the', 'for', 'memory', 'best', 'medicine', 'is']


In [5]:
word2index = {word : idx+1 for idx, word in enumerate(vocab)}
word2index["<unk>"] = 0
print(word2index)

{'Repeat': 1, 'the': 2, 'for': 3, 'memory': 4, 'best': 5, 'medicine': 6, 'is': 7, '<unk>': 0}


In [6]:
index2word = {idx : word for word, idx in word2index.items()}
print(index2word)

{1: 'Repeat', 2: 'the', 3: 'for', 4: 'memory', 5: 'best', 6: 'medicine', 7: 'is', 0: '<unk>'}


In [7]:
# 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 함수
def buil_data(sentence, word2index):
    encoded = [word2index[word] for word in sentence]
    input_seq, label_seq = encoded[:-1], encoded[1:]
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)
    return input_seq, label_seq

In [8]:
X, Y = buil_data(sentence, word2index)
print(X)
print(Y)

tensor([[1, 7, 2, 5, 6, 3]])
tensor([[7, 2, 5, 6, 3, 4]])


# 2. 모델 구현하기

In [42]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size)
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x):
        output = self.embedding_layer(x)
        output, hidden = self.rnn(output)
        output = self.linear(output)
        return output.view(-1, output.shape[2])

In [43]:
vocab_size = len(word2index) # <unk> 포함
input_size = 5
hidden_size = 20

In [44]:
model = Net(vocab_size, input_size, hidden_size)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())
nb_epochs = 200

In [24]:
output = model(X)
print(output)
print(output.shape)

tensor([[ 0.1510, -0.1057, -0.2058,  0.0395, -0.0247, -0.2041, -0.0230,  0.0372],
        [ 0.0719,  0.2798, -0.0633,  0.4156,  0.3582,  0.1935,  0.1251,  0.2459],
        [ 0.0893,  0.2253, -0.1257, -0.0484,  0.1712,  0.2547,  0.0961,  0.3699],
        [-0.0636, -0.0543, -0.0725, -0.0672, -0.3806, -0.0295, -0.1496, -0.0218],
        [-0.0685,  0.2305,  0.0437,  0.3823,  0.1842,  0.0773,  0.0769,  0.3030],
        [ 0.1726, -0.0654, -0.2509, -0.0884,  0.2205, -0.0285,  0.0900,  0.0700]],
       grad_fn=<ViewBackward>)
torch.Size([6, 8])


In [35]:
for epoch in range(nb_epochs+1):
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        results = output.argmax(dim = 1)
        print("epoch :", epoch, "loss :\n", loss, "result :", results)

epoch : 0 loss :
 tensor(0.0351, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 20 loss :
 tensor(0.0321, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 40 loss :
 tensor(0.0295, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 60 loss :
 tensor(0.0272, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 80 loss :
 tensor(0.0252, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 100 loss :
 tensor(0.0234, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 120 loss :
 tensor(0.0218, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 140 loss :
 tensor(0.0203, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 160 loss :
 tensor(0.0190, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 180 loss :
 tensor(0.0179, grad_fn=<NllLossBackward>) result : tensor([7, 2, 5, 6, 3, 4])
epoch : 200 loss :
 tensor

In [41]:
print(" ".join([index2word[idx] for idx in results.tolist()]))

is the best medicine for memory
